In [1]:
from __future__ import print_function, division
import os
from io import BytesIO
import bson
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [2]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()

In [10]:
class CdiscountDataset(Dataset):
    """Cdiscount dataset."""

    def __init__(self, offsets_csv, images_csv, bson_file_path, with_label, transform=None):
        self.offsets_df = pd.read_csv(offsets_csv, index_col=0)
        self.images_df = pd.read_csv(images_csv, index_col=0)
        self.bson_file_path = bson_file_path
        self.with_label = with_label
        self.transform = transform

    def __len__(self):
        return len(self.images_df)

    def __getitem__(self, idx):
        image_row = self.images_df.iloc[idx]
        product_id = image_row["product_id"]
        offset_row = self.offsets_df.loc[product_id]

        # Read this product's data from the BSON file.
        bson_file = open(self.bson_file_path, "rb")
        bson_file.seek(offset_row["offset"])
        item_data = bson_file.read(offset_row["length"])

        # Grab the image from the product.
        item = bson.BSON.decode(item_data)
        img_idx = image_row["img_idx"]
        bson_img = item["imgs"][img_idx]["picture"]
        img = io.imread(BytesIO(bson_img))
        
        if self.transform:
            img = self.transform(img)
            
        label = torch.FloatTensor(1)        
        if self.with_label:
            label = torch.FloatTensor([image_row["category_idx"].item()])

        sample = {'img': img, 'label': label}
        return sample

In [11]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        img = transform.resize(sample, (self.output_size, self.output_size))
        return img
    
class ToTensor(object):
    def __call__(self, sample):
        sample = sample.transpose((2, 0, 1))
        return torch.from_numpy(sample)

# train_dataset = CdiscountDataset(
#     offsets_csv="train_offsets.csv",
#     images_csv="dev_train_images.csv",
#     bson_file_path="/mnt/data/cdiscount/train.bson",
#     with_label=True,
#     transform=transforms.Compose([
#         Rescale(128),
#         ToTensor()
#     ])
# )

val_dataset = CdiscountDataset(
    offsets_csv="train_offsets.csv",
    images_csv="dev_val_images.csv",
    bson_file_path="/mnt/data/cdiscount/train.bson",
    with_label=True,
    transform=transforms.Compose([
        Rescale(256),
        ToTensor()
    ])
)

In [12]:
# for i in range(len(val_dataset)):
#     sample = val_dataset[i]
#     img, label = sample['img'], sample['label']
#     print(label)
#     if i == 30:
#         break

In [18]:
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=True, num_workers=6)

In [19]:
import time

time_start = time.clock()
for i_batch, sample_batched in enumerate(val_dataloader):
    print(i_batch)
    if (i_batch == 10):
        break
print('time elapsed %f'%(time.clock() - time_start))

0
1
2
3
4
5
6
7
8
9
10
time elapsed 0.795648
